In [1]:
import pandas as pd

In [2]:
gmlogs = pd.read_csv('data/gamelogs-coaches.csv')

In [3]:
gmlogs.head()

,franchise,team,season,game_num,date,result,coach
0,ATL,ATL,2001,1,2000-10-31,L,krugelo99c
1,ATL,ATL,2001,2,2000-11-02,L,krugelo99c
2,ATL,ATL,2001,3,2000-11-04,L,krugelo99c
3,ATL,ATL,2001,4,2000-11-06,L,krugelo99c
4,ATL,ATL,2001,5,2000-11-07,L,krugelo99c


In [79]:
tenures = pd.read_csv('categorized/tenures-summarized-categorized.csv')

In [80]:
tenures.head()

,slug,coach,coach_id,franchise,seasons,games,wins,losses,min_date,max_date,left_truncated,ending
0,krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,ATL,3,191,69,122,2000-10-31,2002-12-23,True,fired
1,stottte99c|02|04|ATL,Terry Stotts,stottte99c,ATL,2,137,52,85,2002-12-27,2004-04-14,False,fired
2,woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,ATL,6,492,206,286,2004-11-03,2010-04-14,False,fired
3,drewla01c|10|13|ATL,Larry Drew,drewla01c,ATL,3,230,128,102,2010-10-27,2013-04-17,False,fired
4,budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,ATL,5,356,195,161,2013-10-30,2017-12-14,False,NaN


In [81]:
tenures.tail()

,slug,coach,coach_id,franchise,seasons,games,wins,losses,min_date,max_date,left_truncated,ending
213,jordaed01c|03|08|WAS,Eddie Jordan,jordaed01c,WAS,6,421,197,224,2003-10-29,2008-11-22,False,fired
214,tapsced99c|08|09|WAS,Ed Tapscott,tapsced99c,WAS,1,71,18,53,2008-11-25,2009-04-15,False,fired
215,saundfl99c|09|12|WAS,Flip Saunders,saundfl99c,WAS,3,181,51,130,2009-10-27,2012-01-23,False,fired
216,wittmra01c|12|16|WAS,Randy Wittman,wittmra01c,WAS,5,377,178,199,2012-01-25,2016-04-13,False,fired
217,brooksc01c|16|17|WAS,Scott Brooks,brooksc01c,WAS,2,110,64,46,2016-10-27,2017-12-13,False,NaN


In [82]:
endings = tenures[['slug','ending']]

In [106]:
elo = pd.read_csv('elo/elo_teams.csv')

In [108]:
elo.head()

,date,season,team,elo,win,franchise
0,1999-11-02,2000,NYK,1594.8749,True,NYK
1,1999-11-02,2000,VAN,1282.8573,False,VANMEM
2,1999-11-02,2000,ATL,1540.9791,False,ATL
3,1999-11-02,2000,MIL,1518.4995,True,MIL
4,1999-11-02,2000,SEA,1493.8604,True,SEAOKC


In [110]:
alldata = pd.merge(
    gmlogs.rename(
        columns={'coach': 'coach_id',}
    ),
    tenures[['slug','coach','coach_id','franchise','min_date']].rename(
        columns={'min_date': 'date',}
    ),
    how='left'
)\
.fillna(method='ffill')\
.merge(elo[['franchise','date','elo']])\
.merge(endings, how='left')

In [111]:
alldata['cum_games'] = alldata.groupby('slug').date.rank()

In [112]:
alldata['win'] = (alldata.result == 'W')
alldata['cum_wins'] = alldata.groupby('slug').win.cumsum()

In [113]:
alldata['win_pct'] = alldata.cum_wins / alldata.cum_games

In [114]:
alldata['last_game'] = alldata.groupby('slug').apply(lambda x: x.cum_games == x.cum_games.max()).reset_index().set_index('level_1').cum_games

In [115]:
death_event = ['fired', 'contract expired', 'mutual']

In [116]:
def classifydeath(row):
    if row.ending in death_event and row.last_game == True:
        return 1
    else:
        return 0

alldata['death'] = alldata.apply(classifydeath, axis=1)

In [118]:
alldata[alldata.last_game].sample(5)

,franchise,team,season,game_num,date,result,coach_id,slug,coach,elo,ending,cum_games,win,cum_wins,win_pct,last_game,death
22492,VANMEM,MEM,2018,28,2017-12-13,L,bickejb01c,bickejb01c|17|17|VANMEM,J.B. Bickerstaff,1398.884065,NaN,9.0,False,1.0,0.111111,True,0
33106,DET,DET,2014,50,2014-02-08,W,cheekma01c,cheekma01c|13|14|DET,Maurice Cheeks,1411.385900,fired,50.0,True,21.0,0.420000,True,1
2405,MIL,MIL,2013,32,2013-01-05,L,skilesc01c,skilesc01c|08|13|MIL,Scott Skiles,1474.665800,mutual,344.0,False,162.0,0.470930,True,1
30600,DAL,DAL,2018,29,2017-12-14,L,carliri01c,carliri01c|08|17|DAL,Rick Carlisle,1443.986610,NaN,751.0,False,421.0,0.560586,True,0
3484,MIN,MIN,2009,19,2008-12-06,L,wittmra01c,wittmra01c|07|08|MIN,Randy Wittman,1355.622800,fired,143.0,False,38.0,0.265734,True,1
